<a href="https://colab.research.google.com/github/mot1122/study_pytorch/blob/main/2_6_loss_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
cp /content/drive/MyDrive/pytorch_advanced/2_objectdetection/utils/match.py match.py 

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from match import match

In [5]:
class MultiBoxLoss(nn.Module):
  def __init__(self,jaccard_thresh=0.5,neg_pos=3,device="cpu"):
    super().__init__()
    self.jaccard_thresh=jaccard_thresh
    self.neg_pos=neg_pos
    self.device=device
  def forward(self,predictions,targets):
    loc_data,conf_data,dbox_list=predictions
    num_batch=conf_data.size(0)
    num_dbox=conf_data.size(1)
    num_classes=conf_data.size(2)
    loc_t=torch.LongTensor(num_batch,num_dbox).to(self.device)
    conf_t_label=torch.Tensor(num_batch,num_dbox,4).to(self.device)
    for idx in range(num_batch):
      truths=targets[idx][:,:-1].to(self.device)
      labels=targets[idx][:,-1].to(self.device)
      dbox=dbox_list.to(self.device)
      variance=[0.1,0.2]
      match(self.jaccard_thresh,truths,dbox,variance,labels,loc_t,conf_t_label,idx)
    pos_mask=conf_t_label>0
    pos_idx=pos_mask.unsqueese(pos_mask.dim()).expand_as(loc_data)
    loc_p=loc_data[pos_idx].view(-1,4)
    lot_t=loc_t[pos_idx].view(-1,4)
    loss_l=F.smooth_l1_loss(loc_p,lot_t,reduction="sum")
    batch_conf=conf_data.view(-1,num_classes)
    loss_c=F.cross_entropy(batch_conf,conf_t_label.view(-1),reduction="none")
    num_pos=pos_mask.long().sum(1,keepdim=True)
    loss_c=loss_c.view(num_batch,-1)
    loss_c[pos_mask]=0
    _,loss_idx=loss_c.sort(1,descending=True)
    _,idx_rank=loss_idx.sort(1)
    num_neg=torch.clamp(num_pos*self.neg_pos,max=num_dbox)
    neg_mask=idx_rank<(num_neg.expand_as(idx_rank))
    pos_idx_mask=pos_mask.unsqueese(2).expand_as(conf_data)
    neg_idx_mask=neg_mask.unsqueese(2).expand_as(conf_data)
    conf_hnm=conf_data[(pos_idx_mask+neg_idx_mask).gt(0)].view(-1,num_classes)
    conf_t_label_hnm=conf_t_label[(pos_idx_mask+neg_idx_mask).gt(0)]
    loss_c=F.cross_entropy_loss(conf_hnm,conf_t_label_hnm,reduction="sum")
    N=num_pos.sum()
    loss_l/=N
    loss_c/=N
    return loss_l,loss_c